# Introduction

(work in process...)

## 1. Construction of E.coli strains with different auxotrophic behaviors

**global medium context**:\
anaerobic	 minimal	 conditions, glucose as sole carbon source. \
    glucose exchange >= -10 mmol/gDW/h, \
    minimal salt exchange >= -1000 mmol/gDW/h, \
    other exchange fluxes = 0.	\
    all reactions <= 1000 mmol/gDW/h, \
    ATP maintenance flux = 8.39 mmol/gDW/h.

In [ ]:
## Import E.coli model ##

import cobra
from cobra.io import read_sbml_model

# model = read_sbml_model("../data/iJR904.xml")
model = read_sbml_model("../data/iJO1366.xml")
# model = read_sbml_model("../data/iJN1463.xml")

In [ ]:
## Define global medium ##
import pandas as pd
#1. Obtain M9 metabolites from carveme library
medium_lib = pd.read_csv("../data/media_db.tsv",sep="\t")

compound_M9 = medium_lib[medium_lib["medium"] == "M9"]

#2. Obtain M9 exchange reactions
medium_M9 = ["EX_"+str(compound)+"_e" for compound in compound_M9["compound"]]
print(medium_M9)

In [ ]:
#3. Define medium based on M9 salt exchange reactions + glucose exchange
import numpy as np 

lb_global = -1000. #lower bound of minimal salt exchange
ub_global = 1000. #upper bound of all reactions
glucose_initial = 10.0
medium = dict(zip(medium_M9,np.repeat( - lb_global,len(medium_M9))))

medium["EX_o2_e"] = 2.0 #anaerobic condition
medium["EX_glc__D_e"] = glucose_initial

print(medium)
#assign medium to global model
model.medium = medium

#constrain maintenance ATP flux as 8.39
model.reactions.get_by_id("ATPM").lower_bound = 8.39
model.reactions.get_by_id("ATPM").upper_bound = 8.39
#default ATPM == 0.92

In [4]:
## Knockout genes to construct different auxotrophic strains & 
## Constrain lower bound of exchange reactions to impose amino acid leakiness ##

from copy import deepcopy 
import pandas as pd

#1. get list of genes to be knocked out for each auxotrophic phenotype
amino_df = pd.read_csv("../data/amino_gene_knockout.tsv",sep="\t")

#select only one pair of amino acid as an example (construction based on more amino acid might require HPC)
amino_df = amino_df.iloc[[9,11]]
# amino_df = amino_df.iloc[[1,10]]
# amino_df = amino_df.iloc[[0,1]]
amino_df

,Amino acid,Gene knockouts,Auxotrophy
9,ile,b3770,EX_ile__L_e
11,lys,b2838,EX_lys__L_e


In [ ]:
#2. knockout process

#initialization 
strains = {} #dictionary to store modified strains, keyed by pair of amino acid considered

for index1,row1 in amino_df.iterrows(): #loop through the first amino acid that strain may be auxotrophic to
    amino1 = row1[0]
    genes1 = row1[1].split(",")
    
    for index2,row2 in amino_df.loc[index1:].iterrows(): #loop through the second amino acid that strain may be auxotrophic to
        amino2 = row2[0]
        if amino2 == amino1: #exclude the same amino acid
            next
            
        else:
            genes2 = row2[1].split(",")

            #define phenotype 00 (auxotrophic to both) for this pair of amino acid
            key00 = str(amino1) + "_" + str(amino2) + "_" + "00"
            strains[key00] = deepcopy(model)
            
            key01 = str(amino1) + "_" + str(amino2) + "_" + "01"
            strains[key01] = deepcopy(model)
            #define phenotype 10 (auxotrophic to the first) for this pair of amino acid
            key02 = str(amino1) + "_" + str(amino2) + "_" + "02"
            strains[key02] = deepcopy(model)   
            #define phenotype 01 (auxotrophic to both) for this pair of amino acid
            key10 = str(amino1) + "_" + str(amino2) + "_" + "10"
            strains[key10] = deepcopy(model)
            #define phenotype 11 (leaky to both) for this pair of amino acid
            key20 = str(amino1) + "_" + str(amino2) + "_" + "20"
            strains[key20] = deepcopy(model)
            #define phenotype 22 (leaky to both) for this pair of amino acid
            key22 = str(amino1) + "_" + str(amino2) + "_" + "22"
            strains[key22] = deepcopy(model)
            
            
            for gene in genes1: 
                #knock out relevant genes for strains auxotrophic to the 1st amino acid
                strains_00 = getattr(strains[key00].genes, gene)
                strains_00.knock_out()
                
                strains_01 = getattr(strains[key01].genes, gene)
                strains_01.knock_out()

                strains_02 = getattr(strains[key02].genes, gene)
                strains_02.knock_out()

            print(f"for strain auxotrophic to {amino1} ({key01},{key02}), genes {genes1} are knocked out.")

            for gene in genes2: #knock out relevant genes for strains auxotrophic to the 2nd amino acid
                strains_00 = getattr(strains[key00].genes, gene)
                strains_00.knock_out()                

                strains_10 = getattr(strains[key10].genes, gene)
                strains_10.knock_out()

                strains_20 = getattr(strains[key20].genes, gene)
                strains_20.knock_out()
            
                
            print(f"for strain auxotrophic to {amino2} ({key20},{key10}), genes {genes2} are knocked out.")
            
print("\n Knockout finished, the modified strains are:",strains)

## 2. Monoculture Tests - FBA growth under different metabolite secretion/uptake intensities

In [6]:
import numpy as np
#create dataframe to vary leakiness level of selected amino acid pair
start = 0.0
stop = 0.2625
interval = 0.0125
# start = 0.0
# stop = 5.05
# interval = 0.05
leakiness_range = np.arange(start,stop, interval, dtype=float)
leakiness_range = np.round(leakiness_range, decimals=4)
leakiness_data = {"ile_leakiness": leakiness_range, "lys_leakiness": leakiness_range}

leakiness_data
leakiness_df = pd.DataFrame(leakiness_data)

In [7]:
def get_additional_nutrient(concentration, Vmax, K):
    """Use external concentrations to bound the uptake flux of nutritious metabolites."""
    new_max_import = - Vmax * concentration /( K + concentration) #arbitrary value imitating shape of monod equation
    if abs(new_max_import) >= concentration:
        return - concentration #negative value, will be added to lower bound
    else:
        return new_max_import

In [8]:
def one_FBA(tmp_model,sec_rxn,up_rxn,sec,up,glucose=None):
    with tmp_model:
        #secretion level
        if glucose == None:
            glc = 10.
        else:
            glc = glucose
        tmp_model.reactions.get_by_id(sec_rxn).lower_bound = sec
        tmp_model.reactions.get_by_id(up_rxn).lower_bound = - up - 0.0001
        tmp_model.reactions.get_by_id(up_rxn).upper_bound = - up
        tmp_model.reactions.get_by_id("EX_glc__D_e").lower_bound = - glc
        try:
            growth = tmp_model.optimize().objective_value
            glc_flux = tmp_model.optimize().fluxes["EX_glc__D_e"]
        except:
            growth = -1.
            glc_flux = 0.
    return growth,glc_flux
        
        

In [56]:
import pandas as pd
import warnings
from tqdm import tqdm
os_experiment = []
warnings.filterwarnings("error")
mult = 1
overall_progress = tqdm(total=len(leakiness_df)**2, desc="Overall Progress")
for leak1 in leakiness_df["ile_leakiness"].tolist():
    for leak2 in leakiness_df["leu_leakiness"].tolist():
        key02 = "ile_leu_02"
        growth_02 = one_FBA(strains[key02],"EX_leu__L_e","EX_ile__L_e",leak2*mult,leak1)
        data_02 = (
        key02,
        leak2*mult,
        leak1,
        growth_02       
    ) 
        os_experiment.append(data_02)
        
        key20 = "ile_leu_20"
        growth_20 = one_FBA(strains[key20],"EX_ile__L_e","EX_leu__L_e",leak1*mult,leak2)
        data_20 = (
        key20,
        leak1*mult,
        leak2,
        growth_20       
    ) 
        os_experiment.append(data_20) 
        overall_progress.update(1)
overall_progress.close()
os_experiment_df = pd.DataFrame(os_experiment, columns = ["key_strain","sec","up","growth"])


Overall Progress: 100%|██████████████████████████████████████████████████████████████████████| 441/441 [00:41<00:00, 10.72it/s]


In [ ]:
os_experiment_df.to_csv(f"../results/csv/one_shot/compare_sec_up_growth_{start}_{stop}_{interval}.csv")